In [ ]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 97.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoProcessor, AutoModelForVision2Seq
import torch
from PIL import Image

torch.cuda.empty_cache()

model_id = "unsloth/Qwen2.5-VL-7B-Instruct-unsloth-bnb-4bit"
model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=torch.float16
)
processor = AutoProcessor.from_pretrained(model_id)

# 안전한 토큰 설정
eos_id = processor.tokenizer.eos_token_id
im_end_id = processor.tokenizer.convert_tokens_to_ids("<|im_end|>")
stop_ids = [t for t in [eos_id, im_end_id] if t is not None]

if model.generation_config.eos_token_id is None:
    model.generation_config.eos_token_id = eos_id
if model.generation_config.pad_token_id is None:
    # Qwen 계열은 pad_token이 없을 수 있어 eos로 대체
    model.generation_config.pad_token_id = eos_id

def chat(messages, max_new_tokens=768, temperature=0.7, top_p=0.9):
    # Qwen2.5 전용 템플릿
    prompt = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    # 이미지 로딩 (RGB 권장)
    imgs = []
    for msg in messages:
        for c in msg["content"]:
            if c["type"] == "image":
                imgs.append(Image.open(c["image"]).convert("RGB"))

    # 텐서화
    inputs = processor(
        text=prompt,
        images=imgs if imgs else None,
        return_tensors="pt"
    ).to(model.device)

    input_len = inputs["input_ids"].shape[-1]

    # 생성
    gen_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,          # 목록형 설명이 끊기지 않도록 샘플링 권장
        temperature=temperature,
        top_p=top_p,
        eos_token_id=stop_ids,   # eos 혹은 <|im_end|> 도달 시 종료
        pad_token_id=model.generation_config.pad_token_id
    )

    # "생성된 부분만" 디코드
    new_tokens = gen_ids[0, input_len:]
    text = processor.decode(new_tokens, skip_special_tokens=True).strip()

    return text

# 시나리오 1
conversation = [{
    "role": "user",
    "content": [
        {"type": "image", "image": "/content/사진 5.png"},
        {"type": "text", "text": "I want to pay."}
    ]
}]

answer = chat(conversation)
print(answer)

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2242: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/6.90G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

It looks like you're in an office or study environment with various items on your desk, including papers, a fan, a cup, and a tissue box. If you need to pay for something, you can use the card that is partially visible on the desk. However, if you're referring to paying for services or products, it would be best to contact the relevant person or service provider directly. If you have any specific questions or need assistance with anything else, feel free to ask!


In [ ]:
# 시나리오 2
conversation = [{
    "role": "user",
    "content": [
        {"type": "image", "image": "/content/사진 6.png"},
        {"type": "text", "text": "My mouth smells weird."}
    ]
}]

answer = chat(conversation)
print(answer)

It's possible that the smell you're noticing is coming from the items on your desk, such as the perfume or other personal care products. Perfumes can sometimes have a strong scent that lingers on surfaces and in the air. If the smell is persistent, it might be worth cleaning your desk and checking for any lingering scents from the products you use.

If the smell is still there after cleaning, it could be coming from something else in your environment. It's important to note that strong odors can sometimes be unpleasant, but they don't necessarily indicate a health issue unless they are accompanied by other symptoms like a headache or dizziness.

If the smell is bothering you, try opening a window or using an air freshener to help clear the air. If the problem persists, it might be best to consult a professional to rule out any underlying issues.
